# Capturando todos os jogos do campeonato brasileiro de 2021

In [1]:
pip install python-dateutil

Note: you may need to restart the kernel to use updated packages.


# Importando Bibliotecas

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
from datetime import date 
from unicodedata import normalize as nm
from dateutil import parser

# Código

In [3]:
##Padronizar mês da data do Jogo.
def retMes(mes):
    if mes == 'Janeiro':
        return '01'
    elif mes == 'Fevereiro':
        return '02'
    elif mes == 'Março':
        return '03'
    elif mes == 'Abril':
        return '04'
    elif mes == 'Maio':
        return '05'
    elif mes == 'Junho':
        return '06'
    elif mes == 'Julho':
        return '07'
    elif mes == 'Agosto':
        return '08'
    elif mes == 'Setembro':
        return '09'
    elif mes == 'Outubro':
        return '10'
    elif mes == 'Novembro':
        return '11'
    elif mes == 'Dezembro':
        return '12'

In [4]:
url = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
#No Total, existem 380 partidas e 20 times disputando, logo para cada rodada do campeonato ocorrem 10 partidas.
#Armazenando jogos
lista_jogos =[]

for ano in range(2019,2021):
    for numero_partida in range(1,381):
        url_jogo = url+'/'+str(ano)+'/'+str(numero_partida)
        x = requests.get(url_jogo)
        soup = BeautifulSoup(x.content,'html.parser')
        
        #Numero do Jogo + rodada + turno
        str_jogo = soup.find(class_='color-white block text-1')
        num_jogo = int(str_jogo.get_text().strip().replace('Jogo: ',''))
        
        if (num_jogo%10 ==0):
            rodada = int(int(num_jogo)/10)
        else:
            rodada = int((int(num_jogo)//10)+1)
        if num_jogo <=190:
            turno = 1
        else:
            turno = 2
        
        
        #Localização do jogo
        local_aux = soup.find_all(class_='text-2 p-r-20')
        estadio = local_aux[0].get_text().split(" - ")[0].strip()
        cidade = local_aux[0].get_text().split(" - ")[1].strip()
        estado = local_aux[0].get_text().split(" - ")[2].strip()
        
        #Data do jogo
        data_soup = soup.find_all(class_='text-2 p-r-20')
        data = data_soup[1].get_text().split(",")[1].strip()
        #Formatando data
        aux_data = data.split(" de ")
        dia = aux_data[0]
        #formatando mes
        mes = aux_data[1]
        mes = retMes(mes)
        ano = aux_data[2]
        
        data_completa = datetime.date(int(ano),int(mes),int(dia))
        data_completa = data_completa.strftime("%d-%m-%Y")
        
        #Pegando nomes dos mandantes e visitantes
        partida_times = soup.find_all(class_='time-nome color-white')
        mandante = partida_times[0].get_text().split(" - ")[0].strip()
        visitante = partida_times[1].get_text().split(" - ")[0].strip()
        
        #Pegando resultados da partida
        resultado_partida = soup.find_all(class_='time-gols block hidden-sm hidden-md hidden-lg')
        mandante_gols = int(resultado_partida[0].get_text())
        visitante_gols = int(resultado_partida[1].get_text())
        gols_partida = mandante_gols + visitante_gols
        placar = str(str(mandante_gols) + '-'+ str(visitante_gols))
        
        if mandante_gols == visitante_gols:
            resultado = 'Empate'
            resultado_mandante = 'Empate'
            resultado_visitante = 'Empate'
        elif mandante_gols > visitante_gols:
            resultado = 'Mandante'
            resultado_mandante = 'Vitoria'
            resultado_visitante = 'Derrota'
        
        else:
            resultado = 'Visitante'
            resultado_mandante = 'Derrota'
            resultado_visitante = 'Vitoria'
        lista_resultado = [resultado, resultado_mandante, resultado_visitante]
        
        placar = str(mandante_gols) + ' x ' + str(visitante_gols)
        
        lista_dados_partida = [str(ano),rodada, num_jogo, turno, mandante, visitante,
                               estadio, cidade, estado, data_completa, mandante_gols, visitante_gols, gols_partida, 
                               resultado, resultado_mandante, resultado_visitante, placar, url_jogo]
        lista_jogos.append(lista_dados_partida)

print(lista_jogos)

[['2019', 1, 1, 1, 'Grêmio', 'Santos', 'Arena do Grêmio', 'Porto Alegre', 'RS', '28-04-2019', 1, 2, 3, 'Visitante', 'Derrota', 'Vitoria', '1 x 2', 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2019/1'], ['2019', 1, 2, 1, 'Atlético', 'Avaí', 'Independência', 'Belo Horizonte', 'MG', '27-04-2019', 2, 1, 3, 'Mandante', 'Vitoria', 'Derrota', '2 x 1', 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2019/2'], ['2019', 1, 3, 1, 'Ceará', 'Csa', 'Arena Castelão', 'Fortaleza', 'CE', '28-04-2019', 4, 0, 4, 'Mandante', 'Vitoria', 'Derrota', '4 x 0', 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2019/3'], ['2019', 1, 4, 1, 'Palmeiras', 'Fortaleza', 'Allianz Parque', 'Sao Paulo', 'SP', '28-04-2019', 4, 0, 4, 'Mandante', 'Vitoria', 'Derrota', '4 x 0', 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2019/4'], ['2019', 1, 5, 1, 'São Paulo', 'Botafogo', '

# Criando DataFrame

In [8]:
df = pd.DataFrame(lista_jogos, columns=['Temporada','Rodada',
                                   'Número do Jogo','Turno','Mandante',
                                   'Visitante','Estadio','Cidade','UF','Data',
                                   'Gols_Mandante','Gols_Visitante',
                                   'Total_Gols_partida','Resultado partida',
                                   'Rsultado Mandante','Resultado Visitante',
                                   'Placar','URL'])


In [9]:
df

,Temporada,Rodada,Número do Jogo,Turno,Mandante,Visitante,Estadio,Cidade,UF,Data,Gols_Mandante,Gols_Visitante,Total_Gols_partida,Resultado partida,Rsultado Mandante,Resultado Visitante,Placar,URL
0,2019,1,1,1,Grêmio,Santos,Arena do Grêmio,Porto Alegre,RS,28-04-2019,1,2,3,Visitante,Derrota,Vitoria,1 x 2,https://www.cbf.com.br/futebol-brasileiro/comp...
1,2019,1,2,1,Atlético,Avaí,Independência,Belo Horizonte,MG,27-04-2019,2,1,3,Mandante,Vitoria,Derrota,2 x 1,https://www.cbf.com.br/futebol-brasileiro/comp...
2,2019,1,3,1,Ceará,Csa,Arena Castelão,Fortaleza,CE,28-04-2019,4,0,4,Mandante,Vitoria,Derrota,4 x 0,https://www.cbf.com.br/futebol-brasileiro/comp...
3,2019,1,4,1,Palmeiras,Fortaleza,Allianz Parque,Sao Paulo,SP,28-04-2019,4,0,4,Mandante,Vitoria,Derrota,4 x 0,https://www.cbf.com.br/futebol-brasileiro/comp...
4,2019,1,5,1,São Paulo,Botafogo,Morumbi,Sao Paulo,SP,27-04-2019,2,0,2,Mandante,Vitoria,Derrota,2 x 0,https://www.cbf.com.br/futebol-brasileiro/comp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021,38,376,2,Fortaleza,Bahia,Arena Castelão,Fortaleza,CE,09-12-2021,2,1,3,Mandante,Vitoria,Derrota,2 x 1,https://www.cbf.com.br/futebol-brasileiro/comp...
756,2021,38,377,2,Sport,Athletico Paranaense,Arena Pernambuco,São Lourenço da Mata,PE,09-12-2021,1,1,2,Empate,Empate,Empate,1 x 1,https://www.cbf.com.br/futebol-brasileiro/comp...
757,2021,38,378,2,Red Bull Bragantino,Internacional,Nabi Abi Chedid,Braganca Paulista,SP,09-12-2021,1,0,1,Mandante,Vitoria,Derrota,1 x 0,https://www.cbf.com.br/futebol-brasileiro/comp...
758,2021,38,379,2,Juventude,Corinthians,Alfredo Jaconi,Caxias do Sul,RS,09-12-2021,1,0,1,Mandante,Vitoria,Derrota,1 x 0,https://www.cbf.com.br/futebol-brasileiro/comp...


# Exportando para Excel

In [10]:
df.to_excel("CampeonatoBrasileiro_Jogos_CBF_SerieA.xlsx")